In [270]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

from time import time
import numpy as np
import os
import re

In [341]:
t = time()

md_file = '/Users/breedoon/Drive/Minerva/1. CS166 - Modeling and Analysis of Complex Systems/CS166 - LBA project.md'

with open(md_file, 'r') as f:
    md_data = f.read()

In [342]:
# convert wiki links to normal md links
temp_md = 'temp.md'
wikilinks_pattern = re.compile(r'\[\[\s*(?P<target>[^][|]+?)(\s*\|\s*(?P<label>[^][]+))?\s*\]\]')
section_pattern = re.compile(r'^((?:.{0}|.*[^\\]))\#(.+)')
new_md_data = ''
prev_i = 0
for match in wikilinks_pattern.finditer(md_data):
    target, label = match.group(1, 3)
    if label is None:
        label = target
    sections = section_pattern.findall(target)
    if len(sections) != 0:  # referencing a section in this link
        file, section = sections[0]
        section = section.replace(' ', '-').lower()
        target = file + '#' + section
    new_md_data = new_md_data + md_data[prev_i: match.span()[0]] + f"[{label}]({target})"
    prev_i = match.span()[1]

new_md_data = new_md_data + md_data[prev_i:]

with open(temp_md, 'w') as f:
    f.write(new_md_data)

In [343]:
# from flask import Flask, request
#
# temp_svg = 'temp.svg'
# compile_latex_cmd = f"""/opt/homebrew/lib/node_modules/mathjax-node-cli/bin/tex2svg '{{latex}}' >> {temp_svg}"""
#
# app = Flask(__name__)
# app.config["DEBUG"] = True
# app.secret_key = os.urandom(32)
#
#
# # @app.route('/svg', methods=['GET'])
# # def price():
# #     latex = request.args.get('q')
# #     os.system(compile_latex_cmd.format(latex=latex))
# #     with open(temp_svg, 'r') as f:
# #         return f.read()
#
#
# app.run(host='localhost', port=8813)

In [350]:
temp_html = 'temp.html'
# --mathjax doesn't work because wkhtmltopdf just cannot render it no matter what i do
# --mathml displays well in Safari but no in wkhtmltopdf
# --katex also uses javascript so likely won't work in wkhtmltopdf like mathjax
# --gladtex is a pain to set up and still didn't work for some reason
# --webtex=https://latex.codecogs.com/svg.latex?
cmd = f"""pandoc --css=pandoc.css --self-contained --webtex=https://latex.codecogs.com/svg.latex? -f markdown -t html "{temp_md}" -o {temp_html}"""
os.system(cmd)

[WARNING] This document format requires a nonempty <title> element.
  Defaulting to 'temp' as the title.
  To specify a title, use 'title' in metadata or --metadata title="...".


0

In [327]:
# This way footnotes appear at the bottom of the page and no way to attach them to the end of the text
# output_pdf = 'temp.pdf'
# cmd = f"""pandoc -s -V geometry:paperheight=860mm,paperwidth=210mm,margin=0pt --pdf-engine=/Library/TeX/texbin/pdflatex -f markdown -t pdf "{md_file}" -o {output_pdf} """
# os.system(cmd)

In [349]:
# trying to use gladtex to generate svgs locally, didin't work, some error keeps popping up
# temp_htex = 'temp.htex'
# os.system(f"""pandoc --css=pandoc.css -s --gladtex -t html {temp_md} -o {temp_htex}""")
# os.system(f"""gladtex -d image_dir {temp_htex}""")


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/bin/gladtex", line 11, in <module>
    load_entry_point('GladTeX==3.1.0', 'console_scripts', 'gladtex')()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/GladTeX-3.1.0-py3.7.egg/gleetex/__main__.py", line 474, in main
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/GladTeX-3.1.0-py3.7.egg/gleetex/__main__.py", line 295, in run
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/GladTeX-3.1.0-py3.7.egg/gleetex/__main__.py", line 344, in convert_images
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/GladTeX-3.1.0-py3.7.egg/gleetex/cachedconverter.py", line 146, in convert_all
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/GladTeX-3.1.0-py3.7.egg/gleetex/cachedconverter.py", line 209, in _convert_concurrently
  Fi

256

In [ ]:
# cmd = f"""pandoc --pdf-engine='weasyprint' {temp_html} -o {output_pdf}"""
# os.system(cmd)

In [330]:
PTS_IN_MM = 1 / 25.4 * 72


def get_lowest_y_mm(pdf_filepath):
    with open(pdf_filepath, 'rb') as f:
        pages = list(PDFPage.get_pages(f))

        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        if len(pages) > 1:
            print('More than one page found')

        bboxes = []
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            for lobj in layout:
                if isinstance(lobj, LTTextBox):
                    bboxes.append(lobj.bbox)
        page_height = page.mediabox[3]

    lowest_y = np.array(bboxes)[:, [1, 3]].min()  # coordinates begin from lower left
    return (page_height - lowest_y) / PTS_IN_MM  # convert points to m

In [359]:
temp_pdf = 'output.pdf'
if os.path.exists(temp_pdf):
    os.remove(temp_pdf)
pdf_cmd = """wkhtmltopdf -L {ml} -R {mr} -T {mt} -B {mb} --enable-javascript --no-stop-slow-scripts --page-width 210mm --page-height {page_height} --enable-local-file-access {input} {output}"""
# default_margins = dict(ml='1cm', mr='1cm', mt='1cm', mb='1cm')
default_margins = dict(ml='1.5cm', mr='1.5cm', mt='1.5cm', mb='1.5cm')
os.system(pdf_cmd.format(**{**default_margins, 'mb': 0}, input=temp_html, output=temp_pdf, page_height='1000m'))

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


0

In [360]:
lowest_y = get_lowest_y_mm(temp_pdf)
new_height = np.ceil(lowest_y) + float(default_margins['mb'][:-2]) * 10
os.remove(temp_pdf)

# make with new size
os.system(pdf_cmd.format(**default_margins, input=temp_html, output=temp_pdf, page_height=f'{new_height}mm'))

# remove all pages except for 1st (to remove blank pages)
os.system(f"""qpdf --pages {temp_pdf} 1  -- --replace-input {temp_pdf}""")

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


0

In [333]:
print(time() - t)

21.80215096473694
